In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import asf_search as asf

results = asf.product_search('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252-SLC')
reference = results[0]
stack = reference.stack()

In [3]:
pair = asf.Pair(reference, stack[170])

In [4]:
pair.perpendicular

-15

In [5]:
pair.temporal

datetime.timedelta(days=12)

In [6]:
pair.estimate_mean_coherence()

18.315997099347353